Left-over code:

<table><tr>
<td> <img src="BiPlane_Lift_Chord_vs_Gap.JPG" height="500" width="700"> </td>
<td> <img src="Monoplane_vs_Biplane_vs_Triplane.JPG" height="400" width="700"> </td>
</tr></table>

<h3 align = 'center' color=#CC0000>**Figure 1:** Lift vs Gap/Chord (Biplanes) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Figure 2:** Lift vs Angle of Attack (Biplanes & Triplanes)</h3>

In [ ]:
# plot each geometry
width = 5
pyplot.figure(figsize=(width*1.5, width))
pyplot.title('Biplane Geometry (Gap = 0.75)')
pyplot.grid()
pyplot.xlabel('x', fontsize=16)
pyplot.ylabel('y', fontsize=16)
pyplot.plot(x_m, y_m, color='k', linestyle='-', linewidth=2)
pyplot.plot(x_bi_4, y_bi_4, color='k', linestyle='-', linewidth=2)
pyplot.xlim(-0.1, 1.1)
pyplot.ylim(-0.2, 1.0);

In [ ]:

#----------------------------------------------------------------------#

# gap 2: 1.00
A_bi_2 = build_singularity_matrix(A_source_bi_2, B_vortex_bi_2, N)
b_bi4_2 = build_freestream_rhs(panelS_bi_2, freestream_4, N)
#print(A_bi_2)
#print(b_bi4_2)

# solve for the singularity matrices
strengths_bi4_2 = numpy.linalg.solve(A_bi_2,b_bi4_2)

#store the strengths on each panel
for i, panel in enumerate(panelS_bi_2):
    panel.sigma = strengths_bi4_2[i]

# store the circulation density
gamma_bi4_2 = strengths_bi4_2[-2:]

#----------------------------------------------------------------------#

# gap 3: 1.25
A_bi_3 = build_singularity_matrix(A_source_bi_3, B_vortex_bi_3, N)
b_bi4_3 = build_freestream_rhs(panelS_bi_3, freestream_4, N)
#print(A_bi_3)
#print(b_bi4_3)

# solve for the singularity matrices
strengths_bi4_3 = numpy.linalg.solve(A_bi_3,b_bi4_3)

#store the strengths on each panel
for i, panel in enumerate(panelS_bi_3):
    panel.sigma = strengths_bi4_3[i]

# store the circulation density
gamma_bi4_3 = strengths_bi4_3[-2:]

#----------------------------------------------------------------------#

# gap 4: 1.50
A_bi_4 = build_singularity_matrix(A_source_bi_4, B_vortex_bi_4, N)
b_bi4_4 = build_freestream_rhs(panelS_bi_4, freestream_4, N)
#print(A_bi_4)
#print(b_bi4_4)

### Flow Tangency Boundary Condition

Now we will incorporate the flow tangency boundary condition. This will make the body geometry correspond to a dividing streamline, as flow is not permitted to penetrate through the panels. To do this for each panel $i$, we make $u_n=0$ at $(x_{c_i},y_{c_i})$, and we will add a source strength $\sigma$ and a vortex strength $\gamma$ to each panel. This will lead to:

$$
\begin{split}
    \phi(x, y) &= U_\infty x \cos\alpha + U_\infty y \sin\alpha \\
    & + \int_{lower} \frac{\sigma(s)}{2 \pi} \ln \sqrt{(x - \xi(s))^2 + (y - \eta(s))^2} ds  \\
    & + \int_{upper} \frac{\sigma(s)}{2 \pi} \ln \sqrt{(x - \xi(s))^2 + (y - \eta(s))^2} ds \\
        & - \int_{lower} \frac{\gamma(s)}{2 \pi} \tan^{-1} \left( \frac{y - \eta(s)}{x - \xi(s)} \right) ds  \\
    & - \int_{upper} \frac{\gamma(s)}{2 \pi} \tan^{-1} \left( \frac{y - \eta(s)}{x - \xi(s)} \right) ds
\end{split}
$$

Here: 
1. Each panel on each airfoil will have a different source strength $\sigma$.
2. All panels for a given airfoil will share the same vortex strength $\gamma$ for simplicity.
3. $\sigma(s)$ and $\gamma(s)$ can be taken as constants for each panel, as they won't change over the length of the panel.

i.e.: 
$$
\begin{align*}
0 &= V_\infty \cos \left(\alpha-\beta_i\right) + \frac{\sigma_i}{2} \\
&+ \sum_{j=1,j\neq i}^N \frac{\sigma_j}{2\pi} \int_j \frac{\partial}{\partial n_i} \ln \left(\sqrt{(x_{c_i}-x_j(s_j))^2+(y_{c_i}-y_j(s_j))^2} \right) {\rm d}s_j \\
&- \sum_{j=1,j\neq i}^N \frac{\gamma}{2\pi} \int_j \frac{\partial}{\partial n_i} \tan^{-1} \left(\frac{y_{c_i}-y_j(s_j)}{x_{c_i}-x_j(s_j)}\right) {\rm d}s_j
\end{align*}
$$

with: $$\frac{\partial x_{c_i}}{\partial t_i} = -\sin \beta_i$$  

$$\frac{\partial y_{c_i}}{\partial t_i} = \cos \beta_i$$

$$
A^n_{ij} = \frac{1}{2 \pi} \int_0^{l_j} \frac{\left( {x_c}_i - \xi_j(s) \right) \cos\beta_i + \left( {y_c}_i - \eta_j(s) \right) \sin\beta_i}{\left( {x_c}_i - \xi_j(s) \right)^2 + \left( {y_c}_i - \eta_j(s) \right)^2} ds
$$

$$
B^n_{ij} = \frac{1}{2 \pi} \int_0^{l_j} \frac{\left( {y_c}_i - \eta_j(s) \right) \cos\beta_i - \left( {x_c}_i - \xi_j(s) \right) \sin\beta_i}{\left( {x_c}_i - \xi_j(s) \right)^2 + \left( {y_c}_i - \eta_j(s) \right)^2} ds
$$

Let's create functions for the integral, and the source and vortex contributions.